In [37]:
import pandas as pd
import numpy as np 
from bed_reader import open_bed, sample_file
import time
import re
import os
import math 
import itertools as itertools
from itertools import product
import statsmodels.stats.multitest
from scipy.stats import norm
from scipy.sparse import lil_matrix
from scipy.io import savemat, loadmat
from scipy.linalg import inv
# Using R inside python
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.vectors import FloatVector
# Set working directory
os.chdir("E:\\GWAS\\plink_win64_20231211")
# Defining the R script and loading the instance in Python
r = robjects.r
r['source']('sun_cai(2007)_est.R')

# Loading the functions for sun and cai method defined in R.
est_sun_cai_r = robjects.globalenv['epsest.func']   #### function for estimating $\pi$

rej_sun_cai_r = robjects.globalenv['adaptZ.funcnull']

lcfdr_sun_cai_r = robjects.globalenv['adaptZ.funcnulllcfdr']

def locFDRS_GWAS_thread(Z,scov,B,b,pi,tau):
    K = np.size(Z);
    locFDR = np.zeros(K);
    for i in range(K):
        Z_sub = Z[max(i-B,0):(min(i+B,K-1)+1)];
        cov_sub = scov[max(i-B,0):(min(i+B,K-1)+1),max(i-B,0):(min(i+B,K-1)+1)].toarray();
        l = min(i+B,K-1) - max(i-B,0) + 1;
        H = product([0,1], repeat=l);                                           
        sum1 = sum2 = 0;
        for s in H:
            if(s[min(B,i)] == 0): 
                t = ((np.exp(-(1/2)*np.dot(np.inner((Z_sub - np.multiply(s,b)), 
                                         np.linalg.inv(cov_sub+np.multiply(np.diag(s),pow(tau,2)))),
                                  (Z_sub - np.multiply(s,b))))* 
                pow(pi,sum(s)) * pow(1-pi,l-sum(s)))/np.sqrt(np.linalg.det(cov_sub+np.multiply(np.diag(s),pow(tau,2)))));
                sum1 = sum1 + t;
                sum2 = sum2 + t;
            else:
                sum2 = sum2 + ((np.exp(-(1/2)*np.inner(np.dot(Z_sub - np.multiply(s,b), 
                                         np.linalg.inv(cov_sub+np.multiply(np.diag(s),pow(tau,2)))),
                                  (Z_sub - np.multiply(s,b))))* 
               pow(pi,sum(s)) * pow(1-pi,l-sum(s)))/np.sqrt(np.linalg.det(cov_sub+np.multiply(np.diag(s),pow(tau,2)))));
        locFDR[i]= sum1/sum2;
  
  
    return(locFDR);

def rejected(p_val,level):
    oo = np.argsort(p_val);
    ss = np.sort(p_val);
    stat = np.divide(np.cumsum(ss),np.arange(1,np.size(ss)+1,1));
    collection = np.where(stat <= level);
    return(oo[collection]);

### Block Sizes are determined manually so that each block contains ~3500 to ~4000 SNPs, you need to do it on your own for your data 
partition = np.array([3850,4100,3875,4050,3950,3950,4100,3710,3900,3750,3700,4200,4500,4450,4150,3800,4450,3900,3900,3500,3850,4400]); 
for chrno in range(1,23):
    num_snps_block = partition[chrno-1];
    print(num_snps_block);
    # chrno = 18;num_snps_block = 3870;
    filenamebed = f'ImpQcLDpCh{chrno}.bed'
    filenamebim = f'ImpQcLDpCh{chrno}.bim'
    filenamefam = f'ImpQcLDpCh{chrno}.fam'
    
    import pandas as pd
    import numpy as np
    # phen = pd.read_csv("height&id.csv")
    # phen = phen.iloc[:,[1,2]]
    # phenidnonmiss = phen.drop(np.where(np.array(np.transpose(phen.iloc[:,1])).astype('str') == 'nan')[0])
    from bed_reader import open_bed
    with open_bed(filenamebed) as bed3:
        idd = bed3.iid[:];
        snp = bed3.sid[:];
        print(snp.shape)
        print(idd.shape)
    
    with open_bed(filenamebed) as bed:
        idd = bed.iid[:];
        snp = bed.sid[:];
    phen = pd.read_csv("height&id.csv")
    phen = phen.iloc[:,[1,2]]
    phenidnonmiss = phen.drop(np.where(np.array(np.transpose(phen.iloc[:,1])).astype('str') == 'nan')[0])
    common_iid = np.intersect1d(idd,phenidnonmiss['eid'])
    common_iid = common_iid.astype('int64')
    order_common_iid = idd[np.where(np.isin(idd,common_iid)==True)[0]]
    argsort = np.argsort(order_common_iid.astype('int64'))
    phen_filt = phenidnonmiss.loc[phenidnonmiss['eid'].isin(common_iid)];
    height_unordered = phen_filt['50-0.0'] #21001-0.0
    # height_unordered = phen_filt['21001-0.0']
    height = np.array(height_unordered)#[argsort]
    # comn_snps = pd.read_csv("index_common.csv")
    
    bim20 = open(filenamebim,"r");
    bim20 = bim20.readlines();
    func = lambda x: re.split(r'\t+',x[:-1])
    x_func = np.vectorize(func)
    bim20 = np.array([np.concatenate([x_func(x)]) for x in bim20]);
    # bim20
    
    LDbreakbed = f'fourier_ls-chr{chrno}.bed'
    indblock20 = open(LDbreakbed,"r");
    indblock20 = indblock20.readlines();
    func = lambda x: re.split(r'\t+',x[:-1])
    x_func = np.vectorize(func)
    indblock20 = np.array([np.concatenate([x_func(x)]) for x in indblock20]);
    # indblock20
    
    rsidlist = [];
    for indx in indblock20[1:]:
        print(indx);
        idx = np.arange(indx[1].astype('int'),indx[2].astype('int')).astype('<U10');
        rsid = bim20[pd.DataFrame(bim20)[3].isin(idx)][:,1];
        rsidlist.append(rsid);
    
    num_snps = 0;
    for rsid in rsidlist:
        print(np.size(rsid));
        num_snps+=np.size(rsid);
    print(num_snps)
    
    rsidlistn = [];                    
    rsidn = np.array([]);
    for rsid in rsidlist:
        if np.size(rsidn)<num_snps_block:
            rsidn = np.append(rsidn,rsid);
        else:
            rsidlistn.append(rsidn); 
            rsidn = np.array([]);
            rsidn = np.append(rsidn,rsid);
    rsidlistn.append(rsidn);
    for rsidn in rsidlistn:
        print(np.size(rsidn));
    
    from scipy.sparse import lil_matrix
    y=height;
    Z = [];
    ScovJ = [];
    Rsid = [];
    step_size = 1;
    with open_bed(filenamebed) as bed20:
        for rsid in rsidlistn:
            print("done");
            # K = np.size(rsid)
            # X=bed20.read(index=np.s_[np.where(np.isin(idd,common_iid)==True)[0][argsort],np.isin(bed20.sid,rsid)]);
            # X=bed20.read(index=np.s_[np.where(np.isin(idd,common_iid)==True)[0][argsort],
            #              np.where(np.isin(bed20.sid,rsid)==True)[0]]);
            jump_index = np.arange(int(np.size(rsid)/step_size))*step_size;
            filtered_snps = np.where(np.isin(bed20.sid,rsid)==True)[0][jump_index];
            X=bed20.read(index=np.s_[np.where(np.isin(idd,common_iid)==True)[0][argsort],filtered_snps]);
            # X = 2-X;
            print(X.shape);
            snp = rsid[jump_index];
            X = pd.DataFrame(X);
            # prop_nan = np.divide((X.isna().sum()),X.shape[0])
            # idx3 = np.where(prop_nan<0.1)[0]
            # X = X.iloc[:, idx3]
            # snp_id = snp[idx3]
            # X = X.fillna(X.mean());
            idx1 = np.where(X.isna().any() == False)[0];
            X = X.iloc[:,idx1];
            snp_id = snp[idx1];
            idx2 = (X != 0).any(axis=0);
            X = X.loc[:, idx2]
            snp_id = snp_id[idx2]
            # print(X)
            # print(snp_id)
            X = (X - np.mean(X)) #/ np.std(X)
            y = (y-np.mean(y))#/np.std(y)
            XTX = np.matmul(np.transpose(np.array(X)),np.array(X));
            if np.any(np.linalg.eigvals(XTX) <= 1e-10):
                print("not pd")
                continue;
            XTXinv = np.linalg.inv(XTX);
            XTy = np.matmul(np.transpose(np.array(X)),np.array(y));
            betahat = np.matmul(XTXinv,XTy)
            e =  y-np.matmul(np.array(X),betahat)
            est_sigma = np.sqrt(np.dot(e,e)/(np.shape(X)[0]-np.shape(X)[1]))
            # S_inv = np.diag(np.sqrt(1./np.diag(np.array(XTXinv))));
            S_inv = np.diag((np.diag(XTXinv)**-0.5));
            corrzscore = (np.matmul(np.matmul(S_inv,XTXinv),S_inv));
            del XTXinv
            del XTX
            del e
            del XTy
            del X;
            zscore = np.matmul(S_inv,betahat)/est_sigma
            del(S_inv);
            del(betahat);
            Z.append(zscore);
            K = np.size(zscore);
            N=5;
            scovJ = lil_matrix((K, K));
            for n in range((2*N)+1):
                scovJ.setdiag(corrzscore.diagonal(n),n);
                scovJ.setdiag(corrzscore.diagonal(n),-n);
            del(corrzscore);                                                                                                                                                                                                                                                
            ScovJ.append(scovJ);
            del(scovJ);
            Rsid.append(snp_id);
            del(snp_id);
    
    zscore = np.array([]);
    for z in Z:
        zscore=np.append(zscore,z);
    
    pi_est = max(est_sun_cai_r(FloatVector(zscore),0.,1.)+0.)
    print(pi_est)
    
    lcfdrsc = np.array(lcfdr_sun_cai_r(FloatVector(zscore)));
    
    pi_est = max(est_sun_cai_r(FloatVector(zscore),0.,1.)+0.001);
    maxiter =100;
    B = 1;
    est = np.zeros(2);
    for i in range(B):
        est_mat = np.zeros(2*maxiter).reshape(maxiter ,2)
        # z = zscore[10*np.arange(3000)+i];
        z=zscore
        scovest = scovJ = lil_matrix((np.size(z), np.size(z)));
        scovest.setdiag(np.ones(np.size(z)),0);
        b_ini = np.mean(z)/pi_est; tausq_ini = np.amax([0.,(np.var(z)-1-(pow(b_ini,2)*(1-pow(pi_est,2))))/pi_est]); 
        est_mat[0,0] = b_ini; est_mat[0,1] = tausq_ini;
        for iter in 1+np.arange(maxiter-1):
          print(iter);
          p_tau = 1-locFDRS_GWAS_thread(z,scovest,0,b_ini,pi_est,np.sqrt(tausq_ini));
          #pi_next = np.mean(p_tau);
          b_next = (np.sum((p_tau)*z)/max(np.sum((p_tau)),0.01));
          tausq_next = np.amax([0,(np.sum((p_tau)*pow(z-b_next,2))
          /max(np.sum((p_tau)),0.01))-1]);
          b_ini = b_next; tausq_ini = tausq_next;
          est_mat[iter,0] = b_ini; est_mat[iter,1] = tausq_ini;
          if np.amax(abs(est_mat[iter,:]-est_mat[iter-1,:])) < pow(10,-5):
             print(est_mat[iter,:]);
             break;
        est = est+est_mat[iter,:]
    print(est/B);
    
    lcfdr0 = np.array([]);
    lcfdr1 = np.array([]);
    lcfdr2 = np.array([]);
    lcfdr3 = np.array([]);                          # Chr 20: -0.20916179 0.168054 np.sqrt(1.48964483) # Chr 22: -0.15636569,0.1913139,np.sqrt(1.16378251)
    for i in range(len(Rsid)):
        print(i);
        lcfdr0 = np.append(lcfdr0,locFDRS_GWAS_thread(Z[i], ScovJ[i],0, b_next, pi_est, np.sqrt(tausq_next)));
        lcfdr1 = np.append(lcfdr1,locFDRS_GWAS_thread(Z[i], ScovJ[i],1, b_next, pi_est, np.sqrt(tausq_next)));
        lcfdr2 = np.append(lcfdr2,locFDRS_GWAS_thread(Z[i], ScovJ[i],2, b_next, pi_est, np.sqrt(tausq_next)));
        lcfdr3 = np.append(lcfdr3,locFDRS_GWAS_thread(Z[i], ScovJ[i],3, b_next, pi_est, np.sqrt(tausq_next)));
        
    ScovJ_dict = {f'matrix_{idx}': matrix for idx, matrix in enumerate(ScovJ)}
    savemat(f'ScovJp_chr{chrno}.mat', ScovJ_dict)
    np.savez(f'zJp_chr{chrno}.npz', *Z)
    np.savez(f'rsidlistp_chr{chrno}.npz', *Rsid)
    np.savetxt(f'zlcfdrsp_chr{chrno}.csv', np.transpose(np.array([zscore,lcfdrsc,lcfdr0,lcfdr1,lcfdr2,lcfdr3])), delimiter=',')
    
    alpha=0.05;
    print(np.size(rejected(lcfdr0,alpha)))
    print(np.size(rejected(lcfdr1,alpha)))
    print(np.size(rejected(lcfdr2,alpha)))
    print(np.size(rejected(lcfdr3,alpha)))
    print(np.size(rejected(lcfdrsc,alpha))) 

3850
(44315,)
(487409,)
['chr1 ' ' 10583 ' ' 1892607']
['chr1 ' ' 1892607 ' ' 3582736']
['chr1 ' ' 3582736 ' ' 4380811']
['chr1 ' ' 4380811 ' ' 5913893']
['chr1 ' ' 5913893 ' ' 7247335']
['chr1 ' ' 7247335 ' ' 9365199']
['chr1 ' ' 9365199 ' ' 10806984']
['chr1 ' ' 10806984 ' ' 11777841']
['chr1 ' ' 11777841 ' ' 12779466']
['chr1 ' ' 12779466 ' ' 14891511']
['chr1 ' ' 14891511 ' ' 16897730']
['chr1 ' ' 16897730 ' ' 18662899']
['chr1 ' ' 18662899 ' ' 20469149']
['chr1 ' ' 20469149 ' ' 21736588']
['chr1 ' ' 21736588 ' ' 23086883']
['chr1 ' ' 23086883 ' ' 23920590']
['chr1 ' ' 23920590 ' ' 25516845']
['chr1 ' ' 25516845 ' ' 27401867']
['chr1 ' ' 27401867 ' ' 30161881']
['chr1 ' ' 30161881 ' ' 32438685']
['chr1 ' ' 32438685 ' ' 34799758']
['chr1 ' ' 34799758 ' ' 37549183']
['chr1 ' ' 37549183 ' ' 38731847']
['chr1 ' ' 38731847 ' ' 40200567']
['chr1 ' ' 40200567 ' ' 41975327']
['chr1 ' ' 41975327 ' ' 43758457']
['chr1 ' ' 43758457 ' ' 44969183']
['chr1 ' ' 44969183 ' ' 46899501']
['chr1 ' ' 